In [108]:
%matplotlib inline

In [271]:
# mimic feature_eng.utils.get_train_test_sets
import feature_eng.utils as utils
import feature_eng.data_clean as data_clean
import feature_eng.feature_eng as feature_eng

import evaluator

import json
import importlib # import module dynamically
import pandas as pd

# Read config file
config_file = open("config/steps.json", "r")
config = None
try:
    config = json.load(config_file)
finally:
    config_file.close()

steps = config['steps']
models = config['models']
predict = False
if 'predict' in config:
    predict = config['predict']

# Define global variables here
# prop_df, df= None, None

print('Loading data ...')
#TODO(hzn): Add a copy of train or prop dataframe if needed.
train, prop = utils.load_train_data()

print('Cleaning data and feature engineering...')
# Steps that must be done on the global data set.
step0 = steps[0]
# Steps that can be done on training set only, save time when we are not output
# predict data.
step1 = steps[1]
if predict:
    step0 = step0 + step1
for method in step0:
    module_name = method['module']
    module = globals()[module_name]
    method_name = method['method']
    method_to_call = getattr(module, method_name)

    params = method['params']
    args = params['args']
    kwargs = params['kwargs']

    args = list(map(lambda x: globals()[x], args))
    for key, value_name in kwargs.items():
        value = globals()[value_name]
        params[key] = value
    kwargs['df'] = prop

    prop = method_to_call(*args, **kwargs)

# Subset with transaction info
df = train.merge(prop, how='left', on='parcelid')

# Run some feature engineering jobs on trainning set only when not output
# prediction result.
if not predict:
    for method in step1:
        module_name = method['module']
        module = globals()[module_name]
        method_name = method['method']
        method_to_call = getattr(module, method_name)

        params = method['params']
        args = params['args']
        kwargs = params['kwargs']

        args = list(map(lambda x: globals()[x], args))
        for key, value_name in kwargs.items():
            value = globals()[value_name]
            params[key] = value
        kwargs['df'] = df

        df = method_to_call(*args, **kwargs)


print("Spliting data into training and testing...")
# transaction date is needed to split train and test(by ourselves) here.
train_df, test_df = utils.split_by_date(df)
train_df = data_clean.drop_training_only_column(train_df)
test_df = data_clean.drop_training_only_column(test_df)
# 82249 rows
X_train, y_train = utils.get_features_target(train_df)
# 8562 rows
X_test, y_test = utils.get_features_target(test_df)

Loading data ...


/home/peter/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


Cleaning data and feature engineering...
Spliting data into training and testing...


In [52]:
resultdf = pd.read_csv("predict.csv")

In [53]:
resultdf['error'] = abs(resultdf['logerror']-resultdf['predict'])

In [54]:
resultdf.columns = ['pid', 'logerror', 'predict', 'error']

In [55]:
import numpy as np

In [56]:
np.mean(resultdf.sort_values(by='logerror', ascending=False).iloc[100:-100]['error'])

0.05073887556940789

In [57]:
resultdf.sort_values(by='logerror', ascending=False).iloc[10:-10]

,pid,logerror,predict,error
1933,83666,1.1690,0.0040,1.1650
7459,89192,1.1440,0.0067,1.1373
2391,84124,1.1400,0.0081,1.1319
6848,88581,1.1070,0.0083,1.0987
3724,85457,0.9884,0.0024,0.9860
2936,84669,0.9866,0.0094,0.9772
7748,89481,0.9609,0.0019,0.9590
8117,89850,0.9451,0.0019,0.9432
4251,85984,0.9282,0.0094,0.9188
6124,87857,0.9075,0.0038,0.9037


In [58]:
resultdf.sort_values(by='logerror', ascending=False).tail(100)

,pid,logerror,predict,error
5150,86883,-0.3243,0.0068,0.3311
4774,86507,-0.3257,0.0078,0.3335
2292,84025,-0.3285,0.0036,0.3321
2304,84037,-0.3285,0.0036,0.3321
2440,84173,-0.3327,0.0030,0.3357
4855,86588,-0.3327,0.0049,0.3376
4331,86064,-0.3355,0.0090,0.3445
3989,85722,-0.3383,0.0085,0.3468
6370,88103,-0.3383,0.0042,0.3425
6482,88215,-0.3383,0.0034,0.3417


In [68]:
df=resultdf

In [69]:
df.index= df['pid']

In [70]:
pid_series=(resultdf.sort_values(by='error', ascending=False))['pid'].reset_index(drop=True)

In [71]:
pid_series

0       88788
1       87343
2       89775
3       86597
4       86469
5       84574
6       90226
7       85954
8       89068
9       86188
10      82459
11      82389
12      86153
13      85795
14      83666
15      85847
16      89192
17      84124
18      83229
19      88581
20      88632
21      81999
22      89682
23      82073
24      85420
25      87252
26      83224
27      85457
28      89696
29      84669
        ...  
8512    85251
8513    86470
8514    88577
8515    82861
8516    88545
8517    89852
8518    82804
8519    82328
8520    86123
8521    87024
8522    87211
8523    86202
8524    87822
8525    85664
8526    82039
8527    89759
8528    83564
8529    88034
8530    88821
8531    86947
8532    85566
8533    85955
8534    86710
8535    86568
8536    85640
8537    83349
8538    85185
8539    83978
8540    89181
8541    83436
Name: pid, Length: 8542, dtype: int64

In [72]:
df2=X_test.ix[pid_series]

In [73]:
df2['pid']=df2.index

In [74]:
dft=df.merge(df2)

# What if the outliers could be identified by a classifier?

In [250]:
X_train['y'] = y_train

In [251]:
X_train['absy'] = abs(y_train)

In [252]:
X_test['y'] = y_test
X_test['absy'] = abs(y_test)

In [253]:
X_train=X_train.sort_values(by='absy', ascending=False).reset_index()
X_test=X_test.sort_values(by='absy', ascending=False).reset_index()

In [254]:
X_train['label'] = 0
X_train.loc[X_train.index<2000,'label']=-1
X_train.loc[X_train.index>len(X_train)-2000,'label']=1

In [255]:
X_test['label'] = 0
X_test.loc[X_test.index<200,'label']=-1
X_test.loc[X_test.index>len(X_test)-200,'label']=1

In [256]:
X_train = X_train[X_train['label']!=0]
X_test = X_test[X_test['label']!=0]

In [257]:
X_train.head()

,index,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,...,propertyzoningdesc_23,propertyzoningdesc_24,propertyzoningdesc_25,propertyzoningdesc_26,propertyzoningdesc_27,propertyzoningdesc_28,propertyzoningdesc_29,y,absy,label
0,7318,3.0,2.0,0.0,3.0,0.0,2640.0,2640.0,0.0,0.0,...,0,0,0,0,0,0,1,4.737,4.737,-1
1,6416,1.0,3.0,7.0,1.0,0.0,985.0,985.0,0.0,0.0,...,0,0,0,0,0,0,0,-4.605,4.605,-1
2,69228,2.5,3.0,0.0,2.5,943.0,1843.0,1843.0,0.0,0.0,...,0,0,0,0,0,1,0,-4.605,4.605,-1
3,22476,3.0,4.0,4.0,3.0,0.0,3142.0,3142.0,0.0,0.0,...,0,0,0,0,1,0,0,4.520,4.520,-1
4,22475,3.0,4.0,4.0,3.0,0.0,3179.0,3179.0,0.0,0.0,...,0,0,0,0,1,0,0,4.445,4.445,-1


In [258]:
l_train=X_train['label']
l_test=X_test['label']

In [259]:
X_train=X_train.drop(['index','y','absy','label'],1)
X_test=X_test.drop(['index','y','absy','label'],1)

In [260]:
X_test.head()

,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,...,propertyzoningdesc_20,propertyzoningdesc_21,propertyzoningdesc_22,propertyzoningdesc_23,propertyzoningdesc_24,propertyzoningdesc_25,propertyzoningdesc_26,propertyzoningdesc_27,propertyzoningdesc_28,propertyzoningdesc_29
0,2.0,3.0,7.0,2.0,0.0,1417.0,1417.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,1.0,3.0,7.0,1.0,0.0,2075.0,2075.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,2.0,3.0,7.0,2.0,0.0,994.0,994.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,4.0,8.0,7.0,4.0,0.0,4076.0,0.0,0.0,4076.0,0.0,...,0,0,0,0,0,0,0,0,0,1
4,3.0,3.0,4.0,3.0,0.0,1480.0,1480.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [261]:
X_train_m,l_train_m,X_test_m,l_test_m=list(map(lambda x: x.as_matrix(), [X_train, l_train, X_test, l_test]))

In [262]:
len(l_train)

3999

from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import matplotlib.pyplot as plt

pca = PCA(n_components=2)
X_r = pca.fit(X).transform(X)


plt.figure()
colors = ['navy', 'turquoise','darkorange']
target_names = ['0', '1', '-1']

for color, i, target_name in zip(colors, [0,1,-1], target_names):
    plt.scatter(X_r[y == i, 0], X_r[y == i, 1], color=color, alpha=.8,
                label=target_name)
plt.legend(loc='best', shadow=False, scatterpoints=1)
plt.xlim([-0.5E10,0.5E10])
plt.ylim([-0.1E7,0.5E7])
plt.title('PCA')

In [267]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100)
rf.fit(X_train_m, l_train_m)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [268]:
l_pred=rf.predict(X_test_m)

In [269]:
from sklearn.metrics import classification_report

print(classification_report(l_test_m, l_pred))

             precision    recall  f1-score   support

         -1       0.70      0.71      0.70       200
          1       0.70      0.69      0.70       199

avg / total       0.70      0.70      0.70       399



In [270]:
l_pred=rf.predict(X_train_m)

print(classification_report(l_train_m, l_pred))

             precision    recall  f1-score   support

         -1       1.00      1.00      1.00      2000
          1       1.00      1.00      1.00      1999

avg / total       1.00      1.00      1.00      3999



# We have a 70% accuracy to distinguish between the largest and smallest target.

In [312]:
X_train['y'] = y_train
X_train['absy'] = abs(y_train)
X_test['y'] = y_test
X_test['absy'] = abs(y_test)

In [313]:
X_train['label'] = 0
X_train.loc[X_train['absy']>1,'label']=1

In [314]:
X_test['label'] = 0
X_test.loc[X_test['absy']>1,'label']=1

In [315]:
X_train.head()

,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,...,propertyzoningdesc_23,propertyzoningdesc_24,propertyzoningdesc_25,propertyzoningdesc_26,propertyzoningdesc_27,propertyzoningdesc_28,propertyzoningdesc_29,label,y,absy
0,2.0,3.0,4.0,2.0,0.0,1684.0,1684.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0.0276,0.0276
1,3.5,4.0,0.0,3.5,0.0,2263.0,2263.0,0.0,0.0,0.0,...,0,0,0,0,0,1,0,0,-0.1684,0.1684
2,3.0,2.0,4.0,3.0,0.0,2217.0,2217.0,0.0,0.0,0.0,...,0,1,0,0,0,0,0,0,-0.0040,0.0040
3,2.0,2.0,4.0,2.0,0.0,839.0,839.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0.0218,0.0218
4,2.5,4.0,0.0,2.5,0.0,2283.0,2283.0,0.0,0.0,0.0,...,0,0,0,0,0,1,0,0,-0.0050,0.0050


In [316]:
sample_size = len(X_train[X_train['label'] == 1])
zero_indices = X_train[X_train['label'] == 0].index
random_indices = np.random.choice(zero_indices, sample_size, replace=False)
X_train2 = pd.concat([X_train[X_train['label'] == 1], X_train.loc[random_indices]])

In [317]:
sample_size = len(X_test[X_test['label'] == 1])
zero_indices = X_test[X_test['label'] == 0].index
random_indices = np.random.choice(zero_indices, sample_size, replace=False)
X_test2 = pd.concat([X_test[X_test['label'] == 1], X_test.loc[random_indices]])

In [318]:
print(len(X_train2), len(X_test2))

634 50


In [319]:
l_train=X_train2['label']
l_test=X_test2['label']

In [320]:
X_train2=X_train2.drop(['y','absy','label'],1)
X_test2=X_test2.drop(['y','absy','label'],1)

In [321]:
X_test2.head()

,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,...,propertyzoningdesc_20,propertyzoningdesc_21,propertyzoningdesc_22,propertyzoningdesc_23,propertyzoningdesc_24,propertyzoningdesc_25,propertyzoningdesc_26,propertyzoningdesc_27,propertyzoningdesc_28,propertyzoningdesc_29
81999,2.0,3.0,7.0,2.0,0.0,1682.0,1682.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,1
82073,1.0,2.0,7.0,1.0,0.0,1060.0,1060.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
82389,1.0,3.0,7.0,1.0,0.0,1264.0,1264.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
82459,3.0,4.0,4.0,3.0,0.0,2411.0,2411.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,1
83229,1.0,2.0,7.0,1.0,0.0,1159.0,1159.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [322]:
X_train_m,l_train_m,X_test_m,l_test_m=list(map(lambda x: x.as_matrix(), [X_train2, l_train, X_test2, l_test]))

In [323]:
len(l_train)

634

from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import matplotlib.pyplot as plt

pca = PCA(n_components=2)
X_r = pca.fit(X).transform(X)


plt.figure()
colors = ['navy', 'turquoise','darkorange']
target_names = ['0', '1', '-1']

for color, i, target_name in zip(colors, [0,1,-1], target_names):
    plt.scatter(X_r[y == i, 0], X_r[y == i, 1], color=color, alpha=.8,
                label=target_name)
plt.legend(loc='best', shadow=False, scatterpoints=1)
plt.xlim([-0.5E10,0.5E10])
plt.ylim([-0.1E7,0.5E7])
plt.title('PCA')

In [324]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100)
rf.fit(X_train_m, l_train_m)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [310]:
l_pred=rf.predict(X_test_m)

In [311]:
from sklearn.metrics import classification_report

print(classification_report(l_test_m, l_pred))

             precision    recall  f1-score   support

          0       0.64      0.65      0.64        91
          1       0.64      0.64      0.64        91

avg / total       0.64      0.64      0.64       182

